In [ ]:
df = pd.read_csv("Flux-Dev_yolov8_detections.csv", sep="\t")
print(df.columns.tolist())


In [ ]:
import pandas as pd
import json
import ast
import os
import re

INPUT_CSV = "SDXL_1.0_yolov8_detections.csv"
OUTPUT_JSON = "SDXL_1.0_yolov8_detections.json"

def normalize_image_name(name: str) -> str:
    """Keep basename and remove trailing ' (number)' before the extension."""
    base = os.path.basename(str(name).strip())
    # remove ' (123)' right before the extension
    base = re.sub(r"\s*\(\d+\)(?=\.[^.]+$)", "", base)
    return base

def parse_bbox(b):
    """Parse bbox like '[x1, y1, x2, y2]' into a list of floats."""
    if isinstance(b, (list, tuple)):
        return [float(x) for x in b]
    s = str(b).strip()
    if not s:
        return None
    try:
        val = ast.literal_eval(s)
        return [float(x) for x in val]
    except Exception:
        s = s.strip("[]")
        return [float(x) for x in s.split(",")]

# 1) Read CSV (auto-detect delimiter)
df = pd.read_csv(INPUT_CSV, sep=None, engine="python")
df.columns = df.columns.str.strip()

# 2) Basic sanity check
required = {"image_name", "bbox", "score", "label"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"Missing columns in CSV: {sorted(missing)}")

# 3) Clean/normalize fields
df["image_key"] = df["image_name"].apply(normalize_image_name)
df["bbox"] = df["bbox"].apply(parse_bbox)
df["score"] = pd.to_numeric(df["score"], errors="coerce")
df["label"] = pd.to_numeric(df["label"], errors="coerce").astype("Int64")

# drop rows with bad bbox/score/label
df = df.dropna(subset=["bbox", "score", "label"])

# 4) (Optional) drop exact duplicate rows for the same image+box+score+label
df["_bbox_tuple"] = df["bbox"].apply(lambda x: tuple(x))
df = df.drop_duplicates(subset=["image_key", "_bbox_tuple", "score", "label"]).drop(columns="_bbox_tuple")

# 5) Group and build JSON
result = []
for image_name, group in df.groupby("image_key", sort=False):
    boxes = []
    for _, row in group.iterrows():
        boxes.append({
            "box": row["bbox"],
            "score": float(row["score"]),
            "label": int(row["label"])
        })
    result.append({
        "image": image_name,  # normalized name (e.g., 'foo.jpg' not 'foo (1).jpg')
        "boxes": boxes
    })

# 6) Save
with open(OUTPUT_JSON, "w") as f:
    json.dump(result, f, indent=4)

print(f"Wrote {len(result)} images to {OUTPUT_JSON}")
